<a href="https://www.kaggle.com/code/cmdhmn/xgboost?scriptVersionId=248730904" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


In [63]:
train_data = pd.read_csv("/kaggle/input/playground-series-s5e7/train.csv")
test_data = pd.read_csv("/kaggle/input/playground-series-s5e7/test.csv")

In [64]:
train_data['Personality'].value_counts(normalize=True)

Personality
Extrovert    0.739527
Introvert    0.260473
Name: proportion, dtype: float64

In [65]:
for col in test_data.columns:
    if test_data[col].dtype == 'object':
        test_data[col].fillna(test_data[col].mode()[0], inplace=True)
    else:
        test_data[col].fillna(test_data[col].median(), inplace=True)

In [66]:
for col in train_data.columns:
    if train_data[col].dtype == 'object':
        train_data[col].fillna(train_data[col].mode()[0], inplace=True)
    else:
        train_data[col].fillna(train_data[col].median(), inplace=True)

In [67]:
X = train_data.drop(["Personality", "id"],axis=1)
y = train_data["Personality"]

output = pd.DataFrame()
output["id"] = test_data["id"]
test_data.drop(["id"], axis=1, inplace=True)

In [68]:
le = LabelEncoder()
oe = OrdinalEncoder()

In [69]:
y = le.fit_transform(y)

In [70]:
cat = X.select_dtypes(include=["object"]).columns.tolist()
X[cat] = oe.fit_transform(X[cat])

In [71]:
test_data[cat] = oe.transform(test_data[cat])

In [72]:
# from imblearn.over_sampling import SMOTE

# smote = SMOTE(random_state=42)
# X_res, y_res = smote.fit_resample(X, y)

In [73]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.3)

# splits = 5
# skf = StratifiedKFold(n_splits=splits, shuffle=True)

In [74]:
# Balanced data

params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "max_depth": 4,
    "eta": 0.1,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "random_state": 42
}

# models = []
# acc = []

# for i, (train_idx, val_idx) in tqdm(enumerate(skf.split(X_res, y_res)), total=splits, desc="Training.......", colour="blue"):
#     tqdm.write(f"Fold {i + 1}")
#     X_train, X_val = X_res.iloc[train_idx], X_res.iloc[val_idx]
#     y_train, y_val = y_res[train_idx], y_res[val_idx]

#     tqdm.write(f"Fitting Model......")
#     xgbc = XGBClassifier(**params)
#     xgbc.fit(X_train, y_train, verbose=0)

#     tqdm.write(f"Completed")

#     tqdm.write(f"Calculating accuray")

#     pp = xgbc.predict(X_val)

#     facc = accuracy_score(y_val, pp)

#     tqdm.write(f"Accuray: {facc}")

#     acc.append(facc)
#     models.append(xgbc)

#     tqdm.write(" ", end="\r")

# print(f"Mean accuray: {np.mean(acc)}")

In [75]:
# pp = xgbc.predict(X_test)

# print(classification_report(pp, y_test))

In [76]:
test_data.shape

(6175, 7)

In [77]:
# soft voting gives 0.972469

# tp = np.zeros((test_data.shape[0],2))

# for m in models:
#     tp += m.predict_proba(test_data)

# tp /= len(models)

# preds = np.argmax(tp, axis=1)

In [78]:
# hard voting gives 0.972469

# from scipy.stats import mode 

# tp = []

# for m in models:
#     p = m.predict(test_data)
#     tp.append(p)

# tp = np.array(tp)

# preds, _ = mode(tp, axis=0)

In [79]:
# single model

xgbc = XGBClassifier(**params)
xgbc.fit(X, y, verbose=0)


preds = xgbc.predict(test_data)

In [80]:
output["Personality"] = le.inverse_transform(preds)

In [81]:
output.to_csv("submission.csv", index=False)

In [82]:
# output.head()

### **With Imbalanced Data**

| Method        | Accuracy   |
|---------------|------------|
| Soft Voting   | 0.972469 🎯 |
| Hard Voting   | 0.972469 🎯 |
| Single Model  | 0.973279 💥 |

---

### **With Balanced Data (SMOTE)**

| Method        | Accuracy   |
|---------------|------------|
| Soft Voting   | 0.972469 🎯 |
| Hard Voting   | 0.972469 🎯 |
| Single Model  | 0.972469 🎯 |